<a href="https://colab.research.google.com/github/Divyavasanta/hacktoberfest/blob/main/Divya_project1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SQL Generator: Transforming Natural Language Queries into
Database-Ready SQL

In [ ]:
pip install spacy


In [ ]:
!python -m spacy download en_core_web_sm


In [1]:
import spacy
print(spacy.util.get_installed_models())


['en_core_web_sm']


In [2]:
import spacy

In [11]:
nlp = spacy.load('en_core_web_sm')

In [37]:
SCHEMA = {
    "OrderHeader": ["orderId", "orderTypeId", "orderName", "salesChannelEnumId", "orderDate", "entryDate", "orderStatus", "productStoreId", "grandTotal"],
    "OrderItems": ["orderId", "orderItemSeqId", "productId", "quantity", "unitPrice", "statusId"],
    "OrderItemShipGroup": ["orderId", "shipGroupSeqId", "shipmentMethodTypeId", "facilityId", "contactMechId"],
    "OrderContactMech": ["orderId", "contactMechId", "contactMechPurposeTypeId"],
    "PostalAddress": ["contactMechId", "toName", "address1", "city", "postalCode", "countryGeoId"],
    "OrderAdjustment": ["orderAdjustmentId", "orderAdjustmentTypeId", "orderId", "orderItemSeqId", "shipGroupSeqId", "amount", "createdDate"]
}


In [38]:
table_synonyms = {
    "orders": "OrderHeader",
    "order": "OrderHeader",
    "order items": "OrderItems",
    "items": "OrderItems",
    "address": "PostalAddress",
    "adjustments": "OrderAdjustment",
    "order adjustments": "OrderAdjustment",
    "shipping group": "OrderItemShipGroup"
}

In [39]:
def extract_keywords(query):
    doc = nlp(query)
    action = None
    tables = []
    conditions = []

    select_keywords = ["get", "retrieve", "show", "find"]
    condition_keywords = ["where", "in", "for", "with"]

    for token in doc:
        if token.lemma_ in select_keywords:
            action = "SELECT"
        elif token.text.lower() in table_synonyms:
            tables.append(table_synonyms[token.text.lower()])
        elif token.text.lower() in condition_keywords:
            condition_start = token.i + 1
            condition = ' '.join([t.text for t in doc[condition_start:]])
            conditions.append(condition)

    return action, tables, conditions

In [42]:
def generate_sql(action, tables, conditions):
    if not action or not tables:
        return "Invalid query. Could not detect necessary SQL components."

    sql_query = f"{action} * FROM {', '.join(tables)}"

    if conditions:
        sql_query += " WHERE " + " AND ".join(conditions)

    return sql_query

def user_query_to_sql():
    query = input("Enter your query: ")
    action, tables, conditions = extract_keywords(query)
    sql = generate_sql(action, tables, conditions)
    print(f"Generated SQL: {sql}")

user_query_to_sql()

Enter your query: Get all completed orders 
Generated SQL: SELECT * FROM OrderHeader
